In [1]:
#get subset of MUSC records where journals equal those two chosen.

import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element
import pickle
import os
import pandas as pd
import numpy as np
import nltk.data

In [4]:
#on VM
os.chdir('/mnt/mypartition/Desktop2/pubmed_nlp_research/DeepLearningMovies_datasets/')
import KaggleWord2VecUtility

In [2]:
#on local:
import sys
sys.path.append('/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/'

In [17]:
#one-time run
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [8]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

### Getting subset based on 2 journals (with abstracts and no Unicode encoding)

In [6]:
tree = ET.parse('/mnt/mypartition/Desktop2/testXMLwrite.xml')
root = tree.getroot()

In [11]:
#using this file for our example
xml_file = ET.parse('/home/sarahwie/Documents/zip/subset/zip/medline16n0189.xml')
root = xml_file.getroot()

This code subsets the records down to those with the titles we are looking for (as well as those with abstracts).

In [27]:
#Can delete I think for real run
#make new XML file
rootNew = Element('MedlineCitationSet')
children = []

In [28]:
#can delete I think for real run
#replace titles here with those we're using
#titles = ['Physical review letters', 'Stroke; a journal of cerebral circulation']
titles = ['The Journal of biological chemistry','Brain research']

i = 0
for record in root.findall('MedlineCitation'):
    title = record.find('Article').find('Journal').find('Title').text
    
    if title in titles:
        #for test run, also check if abstract
        try:
            abstract = record.find('Article').find('Abstract').find('AbstractText').text
            if type(abstract) != str:
                i = i + 1
                print type(abstract)
            #append to subset file
            children.append(record)
        except AttributeError:
            #dont include if no abstract
            #something random to hold place of indent:
            x = 0

<type 'unicode'>
<type 'unicode'>


In [29]:
rootNew.extend(children)

#create tree and write to file
tree = ET.ElementTree(rootNew)
tree.write("subsetTwoJournals.xml")
        
pickle.dump(rootNew, open('rootTwoJournals.p', 'wb'))

In [20]:
print i

2


In [21]:
#Can delete I think for real run
#size of new subset
print len(rootNew.getchildren())
#verify that this is the correct number: yes

351


/home/sarahwie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  app.launch_new_instance()


### Getting training and testing sets as pandas dataframes

In [30]:
#convert XML subset to pandas dataframe
def iter_docs(author):
    author_attr = author.attrib
    #titles_pos = ['Physical review letters']
    #titles_neg = ['Stroke; a journal of cerebral circulation']
    titles_pos = ['The Journal of biological chemistry']
    titles_neg = ['Brain research']
    for record in author.findall('MedlineCitation'):
        doc_dict = author_attr.copy()
        #doc_dict.update(record.attrib)
        title = record.find('Article').find('Journal').find('Title').text
        doc_dict['journal'] = title
        abstract = record.find('Article').find('Abstract').find('AbstractText').text
        #make sure all abstracts are in string format to avoid later sentence parsing issues (69 in test run were unicode)
        if type(abstract) != str:
            abstract = abstract.encode('utf8')
        doc_dict['abstract'] = abstract
        #add sentiment label based on journal title
        if title in titles_neg:
            doc_dict['sentiment'] = 0
        elif title in titles_pos:
            doc_dict['sentiment'] = 1
        yield doc_dict
        
df = pd.DataFrame(list(iter_docs(rootNew)))
df = df[['sentiment', 'journal', 'abstract']]
print df

     sentiment                              journal  \
0            0                       Brain research   
1            0                       Brain research   
2            0                       Brain research   
3            0                       Brain research   
4            0                       Brain research   
5            0                       Brain research   
6            0                       Brain research   
7            0                       Brain research   
8            0                       Brain research   
9            0                       Brain research   
10           0                       Brain research   
11           0                       Brain research   
12           0                       Brain research   
13           0                       Brain research   
14           0                       Brain research   
15           1  The Journal of biological chemistry   
16           1  The Journal of biological chemistry   
17        

In [31]:
print df.loc[1][1]
#don't know why no quotation marks like when using pandas.to_csv in original w2v inversion
#but seems to work fine so ignore for now
df.loc[0:2]

Brain research


,sentiment,journal,abstract
0,0,Brain research,The dorsolateral quadrant of the lateral septa...
1,0,Brain research,Changes of the internal (d) and external diame...
2,0,Brain research,The neuranatomical location and pharmacologica...


In [33]:
df = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/dfJournals.p', 'rb'))

In [34]:
#split into train and test
#since so little data, consider using cross-validation?
fracTrain = 0.75
nSamples = df.shape[0]
order = np.random.permutation(nSamples) # come up with a random ordering
splitIndex = int(np.round(nSamples*fracTrain))
train1 = df.ix[order[:splitIndex],:]
test1 = df.ix[order[splitIndex:],:]

In [36]:
print train1.shape
print test1.shape

(316, 3)
(106, 3)


In [35]:
#print first record in test set (note has different index value, indexed instead by location (pos. 0))
print test1.iloc[[0]]

     sentiment                              journal  \
360          1  The Journal of biological chemistry   

                                              abstract  
360  The reaction mechanism of melilotate hydroxyla...  


### Parsing Sentences & Building Word2Vec Model

Gets a list of positive review indexes and negative review indexes. Takes those respective reviews and parses them into sentences and appends to a list of positive and negative sentences, respectively. Also does the same for the unlabelled training sentences.

In [37]:
# ****** Split the training set into clean sentences
#
sentences_pos = []  # Initialize an empty list of sentences
sentences_neg = []  # Initialize an empty list of sentences

#here change to include all journal name labels of positive and negative
inxs_pos = np.where(train1['sentiment'] == 1)[0].tolist()
inxs_neg = np.where(train1['sentiment'] == 0)[0].tolist()

print "Parsing sentences from training set"
for inx in inxs_pos:
    review = train1["abstract"].iloc[inx]
    sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

for inx in inxs_neg:
    review = train1["abstract"].iloc[inx]
    sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)    

Parsing sentences from training set


In [35]:
print sentences_pos[0:2]

[[u'we', u'report', u'the', u'first', u'observation', u'of', u'diffractive', u'j', u'psi', u'mu', u'mu', u'production', u'in', u'pp', u'collisions', u'at', u'root', u'square', u's', u'tev'], [u'diffractive', u'events', u'are', u'identified', u'by', u'their', u'rapidity', u'gap', u'signature']]


Now do the same with all sentences to build the vocabulary

In [38]:
# ****** Split the labeled and unlabeled training sets into clean sentences
#
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train1["abstract"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

Parsing sentences from training set


Build Word2Vec model

In [39]:
from gensim.models import Word2Vec
import multiprocessing

## create a w2v learner 
basemodel = Word2Vec(
    workers=multiprocessing.cpu_count(), # use your cores
    iter=3, # iter = sweeps of SGD through the data; more is better
    hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
    )
print basemodel

Word2Vec(vocab=0, size=100, alpha=0.025)


In [40]:
basemodel.build_vocab(sentences) 

Train Word2Vec model:

In [41]:
#model 0 corresponds with Stroke
#and model 1 with Physical Review Letters

from copy import deepcopy
models = [deepcopy(basemodel) for i in range(2)]
models[0].train(sentences_neg, total_examples=len(sentences_neg) )
models[1].train(sentences_pos, total_examples=len(sentences_pos) )

53387

### Applying inversion on the test set

In [42]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

import pandas as pd # for quick summing within doc

def docprob(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

In [43]:
# read in the test set as a list of a list of words
docs = []
for review in test1["abstract"]:
    docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))

In [42]:
print docs[0]

[[u'we', u'report', u'a', u'measurement', u'of', u'the', u'lambda', u'b', u'lifetime', u'in', u'the', u'exclusive', u'decay', u'lambda', u'b', u'j', u'psi', u'lambda', u'in', u'pp', u'collisions', u'at', u'square', u'root', u's', u'tev', u'using', u'an', u'integrated', u'luminosity', u'of', u'fb', u'of', u'data', u'collected', u'by', u'the', u'cdf', u'ii', u'detector', u'at', u'the', u'fermilab', u'tevatron'], [u'using', u'fully', u'reconstructed', u'decays', u'we', u'measure', u'tau', u'lambda', u'b', u'stat', u'syst', u'ps'], [u'this', u'is', u'the', u'single', u'most', u'precise', u'measurement', u'of', u'tau', u'lambda', u'b', u'and', u'is', u'sigma', u'higher', u'than', u'the', u'current', u'world', u'average']]


In [44]:
# get the probs (note we give docprob our test set plus the models)
probs = docprob(docs,models).astype(object)

In [45]:
probs

,0,1
doc,,
0,0.84574,0.15426
1,0.835586,0.164414
2,0.754181,0.245819
3,0.986099,0.0139012
4,0.978837,0.0211629
5,0.916456,0.0835435
6,0.989629,0.0103714
7,0.939087,0.0609127
8,0.797745,0.202255


In [46]:
predictions = np.ones((probs.shape[0]))

In [47]:
print np.shape(predictions)

(106,)


In [48]:
predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model

In [49]:
predictions

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.])

In [50]:
print predictions.shape
print test1["sentiment"].shape

(106,)
(106,)


In [51]:
print np.size(np.where(predictions == test1["sentiment"]))*1./np.size(predictions)

0.481132075472


In [38]:
#testing cross-validation
from sklearn.cross_validation import StratifiedKFold
import numpy as np

y = df['sentiment'].values
skf = StratifiedKFold(y, n_folds=5)
#len(skf)
#print(skf)  
for train_index, test_index in skf:
    print("TRAIN:", train_index, "TEST:", test_index)
    y_train, y_test = y[train_index], y[test_index]

('TRAIN:', array([ 59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
       182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
       195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207,
       208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,
       221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233,
       234, 235, 236, 237, 238, 239, 240, 241, 242, 2

In [6]:
print y_train
print y_test

[0 1]
[0 1]


In [39]:
#issues: need to use the indexes to subset the df pandas dataframe, not y
#also: need to get FIVE sets: 3 for training, 1 for testing and 1 for validation

### Import KaggleWord2VecUtility since didn't work from file

In [33]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences